In [1]:
import numpy as np
import networkx as nx
import scipy.sparse as sp
import sys
import os
import torch
import pickle
import trimesh
from scipy.sparse.linalg.eigen.arpack import eigsh
from utils.util import torch_sparse_tensor, write_pickle_file

In [2]:
from models.networks.smpl import SMPL

In [3]:
smpl = SMPL(pkl_path='pretrains/smpl_model.pkl', isHres=True)

In [4]:
shape = torch.zeros(10).float()
pose_T = torch.zeros(72).float()

In [5]:
verts = smpl(shape[None], pose_T[None])[0]
faces = smpl.faces_hres
print(verts.size())
print(faces.size())

torch.Size([27554, 3])
torch.Size([55104, 3])


In [6]:
mesh = trimesh.Trimesh(vertices=verts, faces=faces, process=False)

In [7]:
mesh.show()

In [8]:
adj = nx.adjacency_matrix(mesh.vertex_adjacency_graph, nodelist=range(verts.shape[0]))

In [9]:
def sparse_to_tuple(sparse_mx):
    """Convert sparse matrix to tuple representation."""
    def to_tuple(mx):
        if not sp.isspmatrix_coo(mx):
            mx = mx.tocoo()
        coords = np.vstack((mx.row, mx.col)).transpose()
        values = mx.data
        shape = mx.shape
        return coords, values, shape

    if isinstance(sparse_mx, list):
        for i in range(len(sparse_mx)):
            sparse_mx[i] = to_tuple(sparse_mx[i])
    else:
        sparse_mx = to_tuple(sparse_mx)

    return sparse_mx

def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()

def preprocess_adj(adj):
    """Preprocessing of adjacency matrix for simple GCN model and conversion to tuple representation."""
    adj_normalized = normalize_adj(adj + sp.eye(adj.shape[0]))
    return sparse_to_tuple(adj_normalized)

def chebyshev_polynomials(adj, k):
    """Calculate Chebyshev polynomials up to order k. Return a list of sparse matrices (tuple representation)."""
    print("Calculating Chebyshev polynomials up to order {}...".format(k))

    adj_normalized = normalize_adj(adj)
    laplacian = sp.eye(adj.shape[0]) - adj_normalized
    largest_eigval, _ = eigsh(laplacian, 1, which='LM')
    scaled_laplacian = (2. / largest_eigval[0]) * laplacian - sp.eye(adj.shape[0])

    t_k = list()
    t_k.append(sp.eye(adj.shape[0]))
    t_k.append(scaled_laplacian)

    def chebyshev_recurrence(t_k_minus_one, t_k_minus_two, scaled_lap):
        s_lap = sp.csr_matrix(scaled_lap, copy=True)
        return 2 * s_lap.dot(t_k_minus_one) - t_k_minus_two

    for i in range(2, k+1):
        t_k.append(chebyshev_recurrence(t_k[-1], t_k[-2], scaled_laplacian))

    return sparse_to_tuple(t_k)

In [10]:
cheb = chebyshev_polynomials(adj, 1)

Calculating Chebyshev polynomials up to order 1...


In [11]:
print(cheb[1])

(array([[    0,     0],
       [    0,  6890],
       [    0,  6891],
       ...,
       [27553, 27548],
       [27553, 27552],
       [27553, 27553]], dtype=int32), array([ 0.33458516, -0.22243086, -0.22243086, ..., -0.22243086,
       -0.22243086,  0.33458516]), (27554, 27554))


In [12]:
adj_mat = torch_sparse_tensor(*cheb[1])
print(adj_mat.shape)

torch.Size([27554, 27554])


In [13]:
adj_mat_hres_info = {'indices': cheb[1][0], 'value': cheb[1][1], 'size': cheb[1][2]}

In [14]:
write_pickle_file('pretrains/adj_mat_hres_info.pkl', adj_mat_hres_info)